### Laboratorium 7 Jak działa GPT?
#### Implemetacja transformera część 2

Imię i nazwisko: ...................

Punktacja:
* 6 pkt. prawidłowa implementacja 
* 2 pkt. wnioski

Rozszerzymy implementację z poprzedniego laboratorium o:
* kodowanie pozycyjne
* warstwę MLP za atencją (co razem daje blok transformera)
* wiele bloków transformera
* strumień resztowy (residual stream)
* normalizację LayerNorm

#### Źródła
* https://youtu.be/kCc8FmEb1nY?si=wYbFi5JB3x-R8375
* https://github.com/karpathy/nanoGPT
* https://arena-chapter1-transformer-interp.streamlit.app/

In [ ]:
import requests
import torch
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# to jest do assertów, proszę zignorować
def set_seeds(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#### Przygotowanie danych uczących
* to samo co do tej pory

In [ ]:
# jako tekst ponownie wykorzystamy HPMOR rozdziały 1-10
# Eliezer Yudkowsky, Harry Potter and the Methods of Rationality https://hpmor.com/
url = "https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/data/hpmor_chapters_1-10.txt?raw=true"
response = requests.get(url)
text = response.text

In [ ]:
len(text)

In [ ]:
characters = sorted(set(text))
vocab_size = len(characters)
idx_to_ch = {i: c for i, c in enumerate(characters)}
ch_to_idx = {c: i for i, c in enumerate(characters)}

In [ ]:
print(vocab_size)

In [ ]:
def code(text):
    return [ch_to_idx[c] for c in text]

def decode(tokens):
    return ''.join(idx_to_ch[i] for i in tokens)

In [ ]:
text_coded = code(text)
# tensor zawierający dane uczące
train = torch.tensor(text_coded)

In [ ]:
def get_batch(data, seq_len=8, batch_size=4):
    '''
    Funkcja zwraca batch danych.
    data (tensor) - dane uczące
    seq_len (int) - długość sekwencji
    batch_size (int) - rozmiar batcha

    X - tensor o kształcie (batch_size, seq_len)
    y - tensor o kształcie (batch_size, seq_len)
    '''
    n = len(data)
    starts = np.random.randint(0, n - seq_len, batch_size)
    X = torch.stack([data[s:s + seq_len] for s in starts])
    y = torch.stack([data[s + 1: s + seq_len + 1] for s in starts])
    return X.to(device), y.to(device)

#### Generacja tekstu

In [ ]:
def generate_text(start_seq, model, max_size, seq_len):
    '''
    Funkcja generuje tekst.
    start_seq (str) - początek tekstu, podany przez użytkownika
    model - sieć neuronowa
    max_size (int) - zadana długość tekstu
    seq_len (int) - długość sekwencji podawanej na wejście modelu
    '''
    for i in range(max_size):
        x = code(start_seq[-seq_len:]) #<- zmiana
        logits = model(torch.tensor([x], device=device)) #<- zmiana []
        probs = F.softmax(logits, dim=-1) #<- zmiana dim=-1
        probs = probs[0, -1, :].cpu().detach().numpy() # <- zmiana
        next_ch = idx_to_ch[np.random.choice(vocab_size, p=probs)]
        start_seq += next_ch
    return start_seq

In [ ]:
def train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size):
    losses = []
    for step in range(n_steps):
        optimizer.zero_grad()
        x, y = get_batch(train, batch_size=batch_size)
        logits = model(x)
        loss = #TODO (uzupełnić z zeszłego tygodnia)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
        if step % 100 == 0:
            print(f"Krok {step}: Loss = {loss.item():.4f}")
    return losses

### Potrzebne modele z zeszłego tygodnia
* proszę skopiować implementację AttentionHead i MultiHeadAttention

In [ ]:
class AttentionHead(nn.Module):
    # TODO

In [ ]:
class MultiHeadAttention(nn.Module):
    # TODO

### Kodowanie pozycyjne (Positional embedding)

* uwaga: na wejściu tej warstwy nie ma tokenów, tylko ich **numery** kolejne
<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/positional_embedding.png?raw=true" alt="Atencja o wielu głowach" width="500">

### Uzupełnić model o kodowanie pozycyjne
* dodać warstwę kodowania pozycyjnego typu nn.Embedding o wymiarach (*seq_len*, *d_model*)
* kodowanie pozycyjne **dodajemy** do wyniku warstwy emb
* na wejściu warstwy kodowania pozycyjnego podajemy numery kolejne sekwencji tokenów (np. [[0, 1, 2, 3, 4, 5, 6, 7]])
* tensor na wejściu ma kształt (*1*, *seq_len*)
* proszę pamiętać o przeniesieniu tensora wejściowego do warstwy pos_emb na urządzenie device


In [ ]:
class MultiHeadModelwithPositionalEmbedding(nn.Module):
    def __init__(self, n_heads, vocab_size, d_head, d_model, seq_len):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model) 
        self.pos_emb =  #TODO
        self.multi_head_attention = MultiHeadAttention(n_heads, d_head, d_model)
        self.linear_out = nn.Linear(d_model, vocab_size)

    def forward(self, X):
        _, seq_len = X.shape # sprawdzamy długość sekwencji w danych wejściowych (model może działać na danych krótszych niż seq_len)
        positional_embedding = #TODO
        x = #TODO
        out = self.multi_head_attention(x)
        logits = self.linear_out(out)
        return logits

In [ ]:
set_seeds(37)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 4
n_heads = 4
n_steps = 2000
model = MultiHeadModelwithPositionalEmbedding(n_heads, vocab_size, d_head, d_model, seq_len).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([-0.1740, -0.0442, -0.2145, -0.1074, -0.0305,  0.0535,  0.0226, -0.1520], device=device), atol=10**(-4))

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Liczba parametrów modelu: {total_params}')

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

In [ ]:
plt.plot(losses);

### Transformer
* tworzymy **TransformerBlock** dodając za atencją:
    * warstwę liniową *d_model* x 4*d_model*
    * funkcję aktywacji GELU
    * warstwę liniową 4*d_model* x *d_model*

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/transformer_block.png?raw=true" alt="Transformer" width="400">

### Zaimplementować klasę MLPBlock
* warstwa liniowa *d_model* x 4*d_model*
* funkcja aktywacji GELU
* warstwa liniowa 4*d_model* x *d_model*


In [ ]:
class MLPBlock(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.linear1 = #TODO
        self.gelu = #TODO
        self.linear2 = #TODO

    def forward(self, x):
        #TODO
        return x 

### Zaimplementować klasę TransformerBlock
* warstwa MultiHeadAttention
* warstwa MLPBlock

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, n_heads, d_head, d_model):
        super().__init__()
        self.multi_head_attention = #TODO
        self.mlp = #TODO

    def forward(self, x):
        #TODO
        return x

### Zaimplementować model z jedną warstwą TransformerBlock - TransformerOneLayerModel
* warstwa embedding
* warstwa kodowania pozycyjnego
* warstwa TransformerBlock
* warstwa liniowa *d_model* x *vocab_size*

**Jedyna różnica w porównaniu do MultiHeadModelwithPositionalEmbedding to zamiana MultiHeadAttention na TransformerBlock**


In [ ]:

class TransformerOneLayerModel(nn.Module):
    def __init__(self, n_heads, vocab_size, d_head, d_model, seq_len):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model) 
        self.pos_emb = #TODO
        self.transformer_block =  #TODO
        self.linear_out = nn.Linear(d_model, vocab_size) 

    def forward(self, X):
        _, seq_len = X.shape
        x = #TODO
        out = #TODO
        logits = self.linear_out(out)
        return logits

In [ ]:
set_seeds(21)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 4
n_heads = 4
n_steps = 2000
model = TransformerOneLayerModel(n_heads, vocab_size, d_head, d_model, seq_len).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([0.1033, 0.1294, 0.1507, 0.1180, 0.1355, 0.1640, 0.1776, 0.1937], device=device), atol=10**(-4))

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Liczba parametrów modelu: {total_params}')

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

In [ ]:
plt.plot(losses);

### Wiele warstw - zaimplementować model TransformerModel
* dodać 3 warstwy TransformerBlock po sobie
* wykorzystać nn.Sequential

In [ ]:
# transformer blocks zamiast transformer block
class TransformerModel(nn.Module):
    def __init__(self, n_heads, vocab_size, d_head, d_model, seq_len):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, d_model) 
        self.pos_emb = #TODO
        self.transformer_blocks = #TODO
        self.linear_out = nn.Linear(d_model, vocab_size)

    def forward(self, X):
        _, seq_len = X.shape
        x = #TODO
        out = #TODO
        logits = self.linear_out(out)
        return logits

In [ ]:
set_seeds(18)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 4
n_heads = 4
n_steps = 2000
model = TransformerModel(n_heads, vocab_size, d_head, d_model, seq_len).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([0.1095, 0.1096, 0.1095, 0.1095, 0.1095, 0.1095, 0.1095, 0.1095], device=device), atol=10**(-4))

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Liczba parametrów modelu: {total_params}')

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

In [ ]:
plt.plot(losses);

### Residual stream
* zmodyfikować transformer block, tak, żeby zawierał residual stream
* $+$ na schemacie oznacza zwykłe dodawanie

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/residual_stream.png?raw=true" alt="Residual stream" width="300">

#### Zmodyfikować TransformerBlock
* dodać residual stream

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, n_heads, d_head, d_model):
        super().__init__()
        self.multi_head_attention = #TODO
        self.mlp = #TODO
        
    def forward(self, x):
        x = #TODO
        x = #TODO
        return x

In [ ]:
set_seeds(73)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 4
n_heads = 4
n_steps = 2000
model = TransformerModel(n_heads, vocab_size, d_head, d_model, seq_len).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([-0.2209, -0.2540, -0.0918,  0.6298, -1.2298,  1.7620,  0.6835, -0.1646], device=device), atol=10**(-4))

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

In [ ]:
plt.plot(losses);

### Layer norm
* uzupełnić Transformer Block o dwie warstwy Layer Norm

<img src="https://github.com/asztyber/jak_dziala_gpt_lab/blob/main/pic/layer_norm.png?raw=true" alt="Layer norm" width="300">

#### Zmodyfikować TransformerBlock
* dodać dwie warstwy Layer Norm w miejscach według schematu (nn.LayerNorm)
* do konstruktora warstwy LayerNorm podajemy rozmiar wymiaru, dla którego ma być zastosowana normalizacja (*d_model*)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, n_heads, d_head, d_model):
        super().__init__()
        self.multi_head_attention = #TODO
        self.mlp = #TODO
        self.ln1 = #TODO
        self.ln2 = #TODO
        
    def forward(self, x):
        #TODO
        return x

In [ ]:
set_seeds(10)
batch_size = 256
seq_len = 8
d_model = 16
d_head = 4
n_heads = 4
n_steps = 2000
model = TransformerModel(n_heads, vocab_size, d_head, d_model, seq_len).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

In [ ]:
x = torch.randint(0, vocab_size, (batch_size, seq_len), device=device)
out = model(x)

In [ ]:
assert out.shape == torch.Size([256, 8, 79])

In [ ]:
assert torch.allclose(out[0, :, 0], torch.tensor([ 0.3888,  0.4468,  0.7320, -0.4866,  0.6700,  0.3762,  0.4792,  0.6207], device=device), atol=10**(-4))

In [ ]:
losses = train_loop(model, optimizer, n_steps, batch_size, seq_len, vocab_size)

In [ ]:
generate_text('T', model, 100, seq_len)

### Wnioski
1. Czy modele będą działać dla danych wejściowych dłuższych niż seq_len? Dlaczego? *Uwaga: w funkcji generate_text dane wejściowe są zawsze przycinane do seq_len*
2. Porównaj wszystkie modele pod względem jakości tekstu, wartości funkcji straty i liczby parametrów
3. (dla chętnych) Można eksperymentować z batch_size, seq_len, d_model, d_head, n_heads, n_steps, lr i liczbą warstw. Jaki najlepszy model udało się uzyskać?

### Dalsze możliwości rozwoju
1. Wykorzystanie danych walidacyjnych (np. kolejnego rozdziału), aby sprawdzić, czy model się nie przeucza
2. Zapis i odczyt checkpointów
3. Zmniejszanie współczynnika uczenia w kolejnych krokach (learning rate decay)
4. Więcej warstw, większe wymiary modelu
5. Większy słownik (tokenizacja!)
6. Obecnie każda głowa jest liczona osobo i wyniki są konkatenowane (a następnie sumowane). Dla wydajności można dodać liczbę głów jako czwarty wymiar. To podjeście jest równoważne matematycznie, jest wydajniejszą implementacją, ale jest trudnejsze do zrozumienia.